![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [5]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [6]:
filename_features = "../Dados/UCI HAR Dataset/features.txt"
filename_labels = "../Dados/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "../Dados/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "../Dados/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "../Dados/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "../Dados/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "../Dados/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "../Dados/UCI HAR Dataset/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], squeeze=True, sep="#")
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'], squeeze=True)
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'], squeeze=True)
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

C:\Users\felip\AppData\Local\Temp\ipykernel_2308\443291619.py:12: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  features = pd.read_csv(filename_features, header=None, names=['nome_var'], squeeze=True, sep="#")
C:\Users\felip\AppData\Local\Temp\ipykernel_2308\443291619.py:15: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'], squeeze=True)
C:\Users\felip\AppData\Local\Temp\ipykernel_2308\443291619.py:19: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'], squeeze=True)


## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [8]:
%%time
clf = DecisionTreeClassifier(random_state=2360873, ccp_alpha=0.001).fit(X_train, y_train)

Wall time: 5.44 s


In [10]:
train_score = clf.score(X_train,y_train)
test_score = clf.score(X_test,y_test)

In [18]:
print('treino '+ str(train_score) )
print('teste ' + str(test_score) )

treino 0.9757889009793254
teste 0.8791991856124872


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [19]:
%%time
prcomp = PCA(n_components=1).fit(X_train)

pc_treino = prcomp.transform(X_train) 
pc_test = prcomp.transform(X_test)

pc_treino.shape

Wall time: 4.22 s


(7352, 1)

In [21]:
n=1

colunas = ['cp'+str(x+1) for x in list(range(n))]

pc_train = pd.DataFrame(pc_treino[:,:n], columns = colunas)
pc_test  = pd.DataFrame( pc_test[:,:n], columns = colunas)

pc_train.head()

,cp1
0,-5.520280
1,-5.535350
2,-5.474988
3,-5.677232
4,-5.748749


In [26]:
%%time
clf_pca = DecisionTreeClassifier(random_state=7, ccp_alpha=0.001).fit(pc_train, y_train)

Wall time: 69.5 ms


In [30]:
train_score_pca= clf_pca.score(pc_train,y_train)
test_score_pca= clf_pca.score(pc_test,y_test)

In [31]:
print('treino '+ str(train_score_pca) )
print('teste ' + str(test_score_pca) )

treino 0.499727965179543
teste 0.45707499151679676


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [45]:
def arvore_pca (n_components):    
    prcomp = PCA(n_components=n_components).fit(X_train)

    pc_treino = prcomp.transform(X_train) 
    pc_test = prcomp.transform(X_test)

    n=n_components

    colunas = ['cp'+str(x+1) for x in list(range(n))]

    pc_train = pd.DataFrame(pc_treino[:,:n], columns = colunas)
    pc_test  = pd.DataFrame( pc_test[:,:n], columns = colunas)

    clf_pca = DecisionTreeClassifier(random_state=7, ccp_alpha=0.001).fit(pc_train, y_train)

    train_score_pca= clf_pca.score(pc_train,y_train)
    test_score_pca= clf_pca.score(pc_test,y_test)

    print('treino '+ str(train_score_pca) )
    print('teste ' + str(test_score_pca) )

In [49]:
%%time
arvore_pca(1)

treino 0.499727965179543
teste 0.45707499151679676
Wall time: 174 ms


In [50]:
%%time
arvore_pca(2)

treino 0.6127584330794341
teste 0.5846623685103495
Wall time: 148 ms


In [51]:
%%time
arvore_pca(5)

treino 0.8460282916213275
teste 0.7885985748218527
Wall time: 180 ms


In [52]:
%%time
arvore_pca(10)

treino 0.8924102285092492
teste 0.8242280285035629
Wall time: 283 ms


In [53]:
%%time
arvore_pca(50)

treino 0.9147170837867247
teste 0.831014591109603
Wall time: 643 ms


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

A acurácia aumentou, mas o tempo de processamento também, visto que com um maior número de dados teremos maior tempo de processamento, para um conjunto de dados muito grande será válido perder um pouco da acurácia para conseguir tornar viável o processamento